In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import math
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
import gc
import wandb
from hypnettorch.data import FashionMNISTData, MNISTData
from hypnettorch.data.special.split_mnist import get_split_mnist_handlers
from hypnettorch.data.special.split_cifar import get_split_cifar_handlers
from hypnettorch.mnets import LeNet, ResNet
from hypnettorch.hnets import HMLP, StructuredHMLP, ChunkedHMLP

from utils.data import get_mnist_data_loaders, get_emnist_data_loaders, randomize_targets, select_from_classes
from utils.visualization import show_imgs, get_model_dot
from utils.others import measure_alloc_mem, count_parameters
from utils.timing import func_timer
from utils.metrics import get_accuracy


from IPython.display import clear_output

torch.set_printoptions(precision=3, linewidth=180)
%env "WANDB_NOTEBOOK_NAME" "main.ipynb"
wandb.login()

In [ ]:
config = {
    "epochs": 2,
    "max_minibatches_per_epoch": 1000,
    "phases": ["hnet->solver", "hnet->hnet->solver", "hnet->solver", "hnet->hnet->solver", "hnet->solver", "hnet->hnet->solver"],
    "data": {
        # "name": "mnist|fmnist",
        "name": "splitmnist",
        "batch_size": 32,
        "data_dir": "data_tmp",
        "num_tasks": 5,
        "num_classes_per_task": 2,
        "validation_size": 0,
    },
    "solver": {
        "use": "lenet",
        "lenet": {
            "arch": "mnist_large",
            "no_weights": True,
        },
        "resnet": {
            "n": 5,
            "k": 1,
            "use_bias": True,
            "no_weights": True,
        },
    },
    "hnet": {
        "lr": 1e-3,
        "reg_lr": 1e-3,
        "model": {
            "layers": [100, 100],
            # "layers": [128, 128],
            "dropout_rate": -1, # hmlp doesn't get images -> need to be added to resnet
        },
        "chunk_emb_size": 16,
        "chunk_size": 4500,
        "cond_in_size": 32,
        "cond_chunk_embs": True,
        "reg_alpha": 1e-2, # L2 regularization of solvers' parameters
        "reg_beta": 8e-1, # regularization against forgetting other contexts (tasks)
        "detach_d_theta": True,
        "reg_clip_grads_max_norm": None,
        "reg_clip_grads_max_value": 1.,
    },
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "wandb_logging": False,
}

print(f"... Running on {config['device']} ...")

In [ ]:
torch.manual_seed(0)
np.random.seed(0)

if config["data"]["name"] == "mnist|fmnist":
    mnist = MNISTData(config["data"]["data_dir"], use_one_hot=True, validation_size=config["data"]["validation_size"])
    fmnist = FashionMNISTData(config["data"]["data_dir"], use_one_hot=True, validation_size=config["data"]["validation_size"])
    data_handlers = [mnist, fmnist]
elif config["data"]["name"] == "splitmnist":
    data_handlers = get_split_mnist_handlers(config["data"]["data_dir"], use_one_hot=True, num_tasks=config["data"]["num_tasks"], num_classes_per_task=config["data"]["num_classes_per_task"], validation_size=config["data"]["validation_size"])
elif config["data"]["name"] == "splitcifar":
    data_handlers = get_split_mnist_handlers(config["data"]["data_dir"], use_one_hot=True, num_tasks=config["data"]["num_tasks"], num_classes_per_task=config["data"]["num_classes_per_task"], validation_size=config["data"]["validation_size"])

assert config["data"]["num_tasks"] == len(data_handlers)

In [ ]:
torch.manual_seed(0)
np.random.seed(0)
# target networks (solvers)
if config["solver"]["use"] == "lenet":
    solver_child = LeNet(in_shape=data_handlers[0].in_shape, num_classes=config["data"]["num_classes_per_task"], **config["solver"]["lenet"]).to(config["device"])
    solver_root = LeNet(in_shape=data_handlers[0].in_shape, num_classes=config["data"]["num_classes_per_task"], **config["solver"]["lenet"]).to(config["device"])
elif config["solver"]["use"] == "resnet":
    solver_child = ResNet(in_shape=data_handlers[0].in_shape, num_classes=config["data"]["num_classes_per_task"], **config["solver"]["resnet"]).to(config["device"])
    solver_root = ResNet(in_shape=data_handlers[0].in_shape, num_classes=config["data"]["num_classes_per_task"], **config["solver"]["resnet"]).to(config["device"])
else:
    raise ValueError(f"Unknown solver: {config['solver']['use']}")

In [ ]:
# hnet = StructuredHMLP(
#     solver.param_shapes,
#     chunk_shapes=[[[16]], [[32]], [[64]], [[16, 16, 3, 3], [16]], [[32, 32, 3, 3], [32]], [[64, 64, 3, 3], [64]], [[10, 64], [10]]],
#     num_per_chunk=[14, 12, 12, 6, 5, 5, 1],
#     chunk_emb_sizes=32,
#     hmlp_kwargs=config["hnet"]["model"],
#     assembly_fct=assembly_fct,
#     uncond_in_size=0, cond_in_size=8, num_cond_embs=2).to(config["device"]
# )
# """
# missing in chunk_shapes:
# [16, 1, 3, 3] [16]
# [32, 16, 3, 3] [32]
# [64, 32, 3, 3] [64]
# """

In [ ]:
torch.manual_seed(0)
np.random.seed(0)

hnet_child = ChunkedHMLP(
    solver_child.param_shapes,
    layers=config["hnet"]["model"]["layers"],
    chunk_size=config["hnet"]["chunk_size"],
    chunk_emb_size=config["hnet"]["chunk_emb_size"],
    cond_chunk_embs=config["hnet"]["cond_chunk_embs"],
    cond_in_size=config["hnet"]["cond_in_size"],
    num_cond_embs=config["data"]["num_tasks"] * 2, # num_tasks * 2 for child hypernetwork
    no_uncond_weights=True,
    no_cond_weights=False,
).to(config["device"])
hnet_child_optim = torch.optim.Adam(hnet_child.conditional_params, lr=config["hnet"]["lr"])

hnet_root = ChunkedHMLP(
    hnet_child.unconditional_param_shapes,
    layers=config["hnet"]["model"]["layers"],
    dropout_rate=config["hnet"]["model"]["dropout_rate"], # only for the root hypernetwork
    chunk_size=config["hnet"]["chunk_size"],
    chunk_emb_size=config["hnet"]["chunk_emb_size"],
    cond_chunk_embs=config["hnet"]["cond_chunk_embs"],
    cond_in_size=config["hnet"]["cond_in_size"],
    num_cond_embs=config["data"]["num_tasks"] * 2, # num_tasks * 2 for child hypernetwork
    no_uncond_weights=False,
    no_cond_weights=False,
).to(config["device"])
hnet_root_optim = torch.optim.Adam(hnet_root.internal_params, lr=config["hnet"]["lr"])

In [ ]:
# with chunking (hypernet -> hypernet -> target net)
# def assembly_fct(list_of_chunks):
#     assert len(list_of_chunks) == 107
#     params_out = [
#         list_of_chunks[0][0],
#         list_of_chunks[1][0],
#         list_of_chunks[2][0],
#         list_of_chunks[0][1],
#         list_of_chunks[2][1],
#         list_of_chunks[1][1]
#     ]
#     to_concat = []
#     for i in range(3, 52 + 3):
#         to_concat.append(list_of_chunks[i][0])
#     params_out.append(torch.cat(to_concat, dim=0))

#     to_concat = []
#     for i in range(52 + 3, 52 + 52 + 3):
#         to_concat.append(list_of_chunks[i][0])
#     params_out.append(torch.cat(to_concat, dim=0))
    
#     return params_out

# hnet_root = StructuredHMLP(
#     hnet_child.param_shapes,
#     chunk_shapes=[[[8],[100]], [[100, 8], [100, 100]], [[420, 100]], [[420]]],
#     num_per_chunk=[2, 1, 52, 52],
#     chunk_emb_sizes=32,
#     hmlp_kwargs=dict(layers=[100, 100]),
#     assembly_fct=assembly_fct,
#     uncond_in_size=0, cond_in_size=8, num_cond_embs=2).to(config["device"]
# )

In [ ]:
# def calc_accuracy(X, y, solver, solver_weights, use_data_from="test"):
#     """Compute the test accuracy for a given dataset (validation)"""
#     # assert use_data_from == "test" or data.num_val_samples > 0, "No validation data available."
#     solver_train = solver.training
#     solver.eval()
#     acc = None

#     with torch.no_grad():
#         if use_data_from == "validation":
#             raise NotImplementedError()
#             # num_correct = 0

#             # for batch_size, X, y, ids in data.val_iterator(config["data"]["batch_size"], return_ids=True):
#             #     X = data.input_to_torch_tensor(X, config["device"], mode='inference')
#             #     y = data.output_to_torch_tensor(y, config["device"], mode='inference')
#             #     y_hat = solver.forward(X, weights=solver_weights)
#             #     num_correct += int(torch.sum(y_hat.argmax(dim=1) == y.argmax(dim=1)).detach().cpu())

#             # acc = num_correct / data.num_val_samples * 100.
#         elif use_data_from == "test":
#                 if solver_weights is not None:
#                     y_hat = solver(X, weights=solver_weights)
#                 else:
#                     y_hat = solver(X)

#                 acc = (y_hat.argmax(dim=-1) == y.argmax(dim=-1)).float().mean() * 100.
#         else:
#             raise ValueError("Unknown data source (use 'test' or 'validation').")

#     solver.train(mode=solver_train)
#     return acc

In [ ]:
def correct_param_shapes(solver, params):
    """Correct the shapes of the parameters for the solver"""
    params_solver = []
    src_param_i = 0
    src_param_start_idx = 0

    for target_param_i, p_shape in enumerate(solver_root.param_shapes):
        curr_available_src_params = params[src_param_i].flatten()[src_param_start_idx:].numel()
        if curr_available_src_params >= math.prod(p_shape):
            params_solver.append(params[src_param_i].flatten()[src_param_start_idx:src_param_start_idx + math.prod(p_shape)].view(p_shape))
            src_param_start_idx += math.prod(p_shape)
        else:
            new_param = torch.zeros(math.prod(p_shape), device=config["device"])
            s = 0

            while math.prod(p_shape) > s:
                curr_available_src_params = params[src_param_i].flatten().numel()
                to_add = params[src_param_i].flatten()[src_param_start_idx:min(curr_available_src_params, src_param_start_idx + (math.prod(p_shape) - s))]
                new_param[s:s + to_add.numel()] = to_add
                s += to_add.numel()

                if s < math.prod(p_shape):
                    src_param_i += 1
                    src_param_start_idx = 0
                else:
                    src_param_start_idx += to_add.numel()

            params_solver.append(new_param.view(p_shape))
    return params_solver

In [ ]:
def calc_delta_theta(optimizer, lr, clip_delta=True, detach=False):
    ret = []
    for g in optimizer.param_groups:
        for p in g["params"]:
            if p.grad is None:
                ret.append(None)
                continue
            if detach:
                ret.append(-lr * p.grad.detach().clone())
            else:
                ret.append(-lr * p.grad.clone())
    return ret

In [ ]:
def get_reg_loss_for_cond(hnet, hnet_prev_params, hnet_optimizer, lr, reg_cond_id, detach_d_theta=False):
    # prepare targets (theta for child nets predicted by previous hnet)
    hnet_mode = hnet.training
    hnet.eval()
    with torch.no_grad():
        theta_child_target = hnet(cond_id=reg_cond_id, weights={"uncond_weights": hnet_prev_params} if hnet_prev_params is not None else None)
    # detaching target below is important!
    theta_child_target = torch.cat([p.detach().clone().view(-1) for p in theta_child_target])
    hnet.train(mode=hnet_mode)
    
    d_theta = calc_delta_theta(hnet_optimizer, lr, detach=detach_d_theta)
    theta_parent_for_pred = []
    for _theta, _d_theta in zip(hnet.internal_params, d_theta):
        if _d_theta is None:
            theta_parent_for_pred.append(_theta)
        else:
            theta_parent_for_pred.append(_theta + _d_theta if detach_d_theta is False else _theta + _d_theta.detach())
    theta_child_predicted = hnet(cond_id=reg_cond_id, weights=theta_parent_for_pred)
    theta_child_predicted = torch.cat([p.view(-1) for p in theta_child_predicted])

    return (theta_child_target - theta_child_predicted).pow(2).sum()

In [ ]:
def get_reg_loss(hnet, hnet_prev_params, hnet_optimizer, curr_cond_id, lr=1e-3, clip_grads_max_norm=1., detach_d_theta=False):
    reg_loss = 0
    for c_i in range(hnet._num_cond_embs):
        if curr_cond_id is not None and c_i == curr_cond_id:
            continue
        reg_loss += get_reg_loss_for_cond(hnet, hnet_prev_params, hnet_optimizer, lr, c_i, detach_d_theta)
    return reg_loss / (hnet._num_cond_embs - (curr_cond_id is not None))

In [ ]:
def infer(X, scenario, hnet_root_cond_id, hnet_child_cond_id, hnet_root, hnet_child, solver_root, solver_child):
    assert scenario != "hnet->hnet->solver" or hnet_child_cond_id is not None, f"Scenario {scenario} requires hnet_child_cond_id to be set"
    
    if scenario == "hnet->solver":
        params_solver = hnet_root.forward(cond_id=hnet_root_cond_id) # root hnet -> params root solver
        y_hat = solver_root.forward(X, weights=correct_param_shapes(solver_root, params_solver))
    elif scenario == "hnet->hnet->solver":
        params_hnet_child = hnet_root.forward(cond_id=hnet_root_cond_id) # root hnet -> params child hnet (only the unconditional ones)
        params_solver = hnet_child.forward(cond_id=hnet_child_cond_id, weights=params_hnet_child)
        y_hat = solver_child.forward(X, weights=params_solver)
    else:
        raise ValueError(f"Unknown inference scenario {scenario}")
    return y_hat, params_solver

In [ ]:
def print_metrics(datasets : dict, hnet_root, hnet_child, solver_root, solver_child, prefix="", skip_phases=[], wandb_run=None, additional_metrics=None):
    # set the models to eval mode and return them to their original mode after
    ms_modes = []
    for m in [hnet_root, hnet_child, solver_root, solver_child]:
        ms_modes.append([m, m.training])
        m.eval()
    wandb_metrics = {}
    
    print(prefix)
    with torch.no_grad():
        for data_name, (hnet_root_cond_id_hnet_solver, hnet_root_cond_id_hnet_hnet_solver, hnet_child_cond_id, dataset) in datasets.items():
            print(data_name)

            # prepare a test batch for calculating loss & getting solver params
            X = dataset.input_to_torch_tensor(dataset.get_test_inputs(), config["device"], mode="inference")
            y = dataset.output_to_torch_tensor(dataset.get_test_outputs(), config["device"], mode="inference")

            hnet_solver_loss, hnet_solver_acc, hnet_hnet_solver_loss, hnet_hnet_solver_acc = np.nan, np.nan, np.nan, np.nan
            if "hnet->solver" not in skip_phases:
                print("    hnet->solver")
                y_hat, params_solver = infer(X, "hnet->solver", hnet_root_cond_id_hnet_solver, None, hnet_root, hnet_child, solver_root, solver_child)
                hnet_solver_loss = F.cross_entropy(y_hat, y).item()
                hnet_solver_acc = (y_hat.argmax(dim=-1) == y.argmax(dim=-1)).float().mean() * 100.
                print(f"        Loss: {hnet_solver_loss:.3f} | Accuracy: {hnet_solver_acc:.3f}")
            
            if "hnet->hnet->solver" not in skip_phases:
                print("    hnet->hnet->solver")
                y_hat, params_solver = infer(X, "hnet->hnet->solver", hnet_root_cond_id_hnet_hnet_solver, hnet_child_cond_id, hnet_root, hnet_child, solver_root, solver_child)
                hnet_hnet_solver_loss = F.cross_entropy(y_hat, y).item()
                hnet_hnet_solver_acc = (y_hat.argmax(dim=-1) == y.argmax(dim=-1)).float().mean() * 100.
                print(f"        Loss: {hnet_hnet_solver_loss:.3f} | Accuracy: {hnet_hnet_solver_acc:.3f}")
            
            wandb_metrics[str(data_name)] = {
                "h->s loss": hnet_solver_loss,
                "h->s acc": hnet_solver_acc,
                "h->h->s loss": hnet_hnet_solver_loss,
                "h->h->s acc": hnet_hnet_solver_acc,
            }
    
    if additional_metrics:
        wandb_metrics.update(additional_metrics)
        for n, v in additional_metrics.items():
            print(f"{n}: {v:.3f}")

    if wandb_run is not None:
        wandb_run.log(wandb_metrics)
    
    for m, mode in ms_modes:
        m.train(mode=mode)

In [ ]:
if config["wandb_logging"]:
    wandb_run = wandb.init(
        project="Hypernets", entity="johnny1188", config=config, group=config["data"]["name"],
        tags=[], notes=f""
    )
    wandb.watch((hnet_root, hnet_child, solver_root, solver_child), log="all", log_freq=100)
else:
    wandb_run = None

In [ ]:
def clip_grads(models, reg_clip_grads_max_norm, reg_clip_grads_max_value):
    if reg_clip_grads_max_norm is not None and reg_clip_grads_max_value is not None:
        print("Warning: both reg_clip_grads_max_norm and reg_clip_grads_max_value are set. Using reg_clip_grads_max_norm.")
    for m in models:
        if reg_clip_grads_max_norm is not None:
            torch.nn.utils.clip_grad_norm_(m.parameters(), reg_clip_grads_max_norm)
        elif reg_clip_grads_max_value is not None:
            torch.nn.utils.clip_grad_value_(m.parameters(), reg_clip_grads_max_value)

#### Training in multitask setting

In [ ]:
torch.manual_seed(0)
np.random.seed(0)
loss_fn = nn.CrossEntropyLoss(reduction="mean")
hnet_root_prev_phase_params = None
log_step = 0

phases = ["hnet->solver", "hnet->hnet->solver", "hnet->solver", "hnet->hnet->solver", "hnet->solver", "hnet->hnet->solver"]
for p_i, phase in enumerate(phases):
    print(f"\n\n.... Starting phase {phase} ...")
    if wandb_run is not None:
        wandb_run.log({"Phase": wandb.Table(columns=["phase", "step"], data=[[phase, log_step]])}) # log what phase the training is in
    for epoch in range(config["epochs"]):
        for i, ((_, m_X, m_y),(_, f_X, f_y)) in enumerate(zip(mnist.train_iterator(config["data"]["batch_size"]), fmnist.train_iterator(config["data"]["batch_size"]))):
            if i > config["max_minibatches_per_epoch"]:
                break

            # Mini-batch of MNIST samples
            m_X = mnist.input_to_torch_tensor(m_X, config["device"], mode="train")
            m_y = mnist.output_to_torch_tensor(m_y, config["device"], mode="train")
            # Mini-batch of FashionMNIST samples
            f_X = fmnist.input_to_torch_tensor(f_X, config["device"], mode="train")
            f_y = fmnist.output_to_torch_tensor(f_y, config["device"], mode="train")

            hnet_root_optim.zero_grad()
            hnet_child_optim.zero_grad()

            # MNIST ------------------------------------------------------------
            if phase == "hnet->solver":
                hnet_root_cond_id = 0
                hnet_child_cond_id = None
            elif phase == "hnet->hnet->solver":
                hnet_root_cond_id = 2
                hnet_child_cond_id = 0
            else:
                raise ValueError(f"Unknown phase {phase}")
            
            y_hat, params_solver = infer(m_X, phase, hnet_root_cond_id=hnet_root_cond_id, hnet_child_cond_id=hnet_child_cond_id, hnet_root=hnet_root, hnet_child=hnet_child, solver_root=solver_root, solver_child=solver_child)
            
            # solvers' params regularization + task loss
            m_loss_solver_params_reg = sum([p.norm(p=2) for p in params_solver]) / len(params_solver)
            m_loss = loss_fn(y_hat, m_y.max(dim=1)[1]) + config["hnet"]["reg_alpha"] * m_loss_solver_params_reg
            m_acc = (y_hat.argmax(dim=-1) == m_y.argmax(dim=-1)).float().mean() * 100.
            m_loss.backward()
            if config["hnet"]["reg_clip_grads_max_norm"] is not None:
                clip_grads([hnet_child, hnet_root], config["hnet"]["reg_clip_grads_max_norm"])
            
            # regularization against forgetting other contexts
            m_loss_reg = config["hnet"]["reg_beta"] * get_reg_loss(hnet_root, hnet_root_prev_phase_params, hnet_root_optim, curr_cond_id=hnet_root_cond_id, lr=config["hnet"]["reg_lr"], detach_d_theta=False)
            m_loss_reg.backward()
            if config["hnet"]["reg_clip_grads_max_norm"] is not None:
                clip_grads([hnet_child, hnet_root], config["hnet"]["reg_clip_grads_max_norm"])
            
            hnet_root_optim.step()
            hnet_child_optim.step()
            hnet_root_optim.zero_grad()
            hnet_child_optim.zero_grad()


            # FashionMNIST ------------------------------------------------------------
            if phase == "hnet->solver":
                hnet_root_cond_id = 1
                hnet_child_cond_id = None
            elif phase == "hnet->hnet->solver":
                hnet_root_cond_id = 3
                hnet_child_cond_id = 1
            else:
                raise ValueError(f"Unknown phase {phase}")
            
            y_hat, params_solver = infer(f_X, phase, hnet_root_cond_id=hnet_root_cond_id, hnet_child_cond_id=hnet_child_cond_id, hnet_root=hnet_root, hnet_child=hnet_child, solver_root=solver_root, solver_child=solver_child)
            
            # solvers' params regularization + task loss
            f_loss_solver_params_reg = sum([p.norm(p=2) for p in params_solver]) / len(params_solver)
            f_loss = loss_fn(y_hat, f_y.max(dim=1)[1]) + config["hnet"]["reg_alpha"] * f_loss_solver_params_reg
            f_acc = (y_hat.argmax(dim=-1) == f_y.argmax(dim=-1)).float().mean() * 100.
            f_loss.backward()
            if config["hnet"]["reg_clip_grads_max_norm"] is not None:
                clip_grads([hnet_child, hnet_root], config["hnet"]["reg_clip_grads_max_norm"])
            
            # regularization against forgetting other contexts
            f_loss_reg = config["hnet"]["reg_beta"] * get_reg_loss(hnet_root, hnet_root_prev_phase_params, hnet_root_optim, curr_cond_id=hnet_root_cond_id, lr=config["hnet"]["reg_lr"], detach_d_theta=False)
            f_loss_reg.backward()
            if config["hnet"]["reg_clip_grads_max_norm"] is not None:
                clip_grads([hnet_child, hnet_root], config["hnet"]["reg_clip_grads_max_norm"])
            
            hnet_root_optim.step()
            hnet_child_optim.step()
            hnet_root_optim.zero_grad()
            hnet_child_optim.zero_grad()


            if i % 100 == 99:
                print_metrics(
                    {"MNIST": (0, 2, 0, mnist), "FashionMNIST": (1, 3, 1, fmnist)},
                    hnet_root, hnet_child, solver_root, solver_child,
                    prefix=f"[{phase} | {epoch}/{config['epochs']} | {i + 1}]\nM: {m_loss_reg:.3f} F: {f_loss_reg:.3f}",
                    # skip_phases=["hnet->hnet->solver"] if p_i == 0 and phase == "hnet->solver" else [],
                    skip_phases=[],
                    wandb_run=wandb_run, additional_metrics={
                        "m_loss_class": m_loss.item() - config["hnet"]["reg_alpha"] * m_loss_solver_params_reg.item(),
                        "f_loss_class": f_loss.item() - config["hnet"]["reg_alpha"] * f_loss_solver_params_reg.item(),
                        "m_acc_class": m_acc,
                        "f_acc_class": f_acc,
                        "m_loss_solver_params_reg": m_loss_solver_params_reg.item(),
                        "f_loss_solver_params_reg": f_loss_solver_params_reg.item(),
                        "m_loss_reg": m_loss_reg.item(),
                        "f_loss_reg": f_loss_reg.item(),
                    }
                )
                print("---")
                log_step += 1
    hnet_root_prev_phase_params = [p.detach().clone() for p_idx, p in enumerate(hnet_root.unconditional_params)]

#### Training in continual learning setting

In [ ]:
hnets_cond_ids = [
    {"hnet->solver": {"hnet_root": task_i, "hnet_child": None}, "hnet->hnet->solver": {"hnet_root": task_i + len(data_handlers), "hnet_child": task_i}}
    for task_i in range(len(data_handlers))
]
datasets_for_eval = {d_i: (cond_ids["hnet->solver"]["hnet_root"], cond_ids["hnet->hnet->solver"]["hnet_root"], cond_ids["hnet->hnet->solver"]["hnet_child"], data_handlers[d_i]) for d_i, cond_ids in enumerate(hnets_cond_ids)}

In [ ]:
torch.manual_seed(0)
np.random.seed(0)
loss_fn = nn.CrossEntropyLoss(reduction="mean")
hnet_root_prev_params = None
log_step = 0
phases = config["phases"]
training_phases_table = wandb.Table(columns=["phase", "dataset_idx", "step"], data=[])

for p_i, phase in enumerate(phases):
    print(f"\n\n.... Starting phase {phase} ...")
    for d_i, data in enumerate(data_handlers):
        hnet_root_prev_params = [p.detach().clone() for p_idx, p in enumerate(hnet_root.unconditional_params)]
        
        if wandb_run is not None:
            training_phases_table.add_data(phase, d_i, log_step)
            wandb_run.log({"Phase": training_phases_table}) # log what phase the training is in

        for epoch in range(config["epochs"]):
            for i, (batch_size, X, y) in enumerate(data.train_iterator(config["data"]["batch_size"])):
                if i > config["max_minibatches_per_epoch"]:
                    break

                X = data.input_to_torch_tensor(X, config["device"], mode="train")
                y = data.output_to_torch_tensor(y, config["device"], mode="train")

                hnet_root_optim.zero_grad()
                hnet_child_optim.zero_grad()

                # select cond_ids for hypernets
                hnet_root_cond_id = hnets_cond_ids[d_i][phase]["hnet_root"]
                hnet_child_cond_id = hnets_cond_ids[d_i][phase]["hnet_child"]
                # generate theta and predict
                y_hat, params_solver = infer(X, phase, hnet_root_cond_id=hnet_root_cond_id, hnet_child_cond_id=hnet_child_cond_id, hnet_root=hnet_root, hnet_child=hnet_child, solver_root=solver_root, solver_child=solver_child)
                
                # solvers' params regularization
                loss_solver_params_reg = sum([p.norm(p=2) for p in params_solver]) / len(params_solver)
                # task loss
                loss_class = loss_fn(y_hat, y)
                loss = loss_class + config["hnet"]["reg_alpha"] * loss_solver_params_reg
                loss.backward(retain_graph=True, create_graph=not config["hnet"]["detach_d_theta"])
                # gradient clipping
                clip_grads([hnet_child, hnet_root], config["hnet"]["reg_clip_grads_max_norm"], config["hnet"]["reg_clip_grads_max_value"])
                
                # regularization against forgetting other contexts
                loss_reg = config["hnet"]["reg_beta"] * get_reg_loss(hnet_root, hnet_root_prev_params, hnet_root_optim, curr_cond_id=hnet_root_cond_id, lr=config["hnet"]["reg_lr"], detach_d_theta=config["hnet"]["detach_d_theta"])
                loss_reg.backward()
                clip_grads([hnet_child, hnet_root], config["hnet"]["reg_clip_grads_max_norm"], config["hnet"]["reg_clip_grads_max_value"])
                
                hnet_root_optim.step()
                hnet_child_optim.step()
                hnet_root_optim.zero_grad()
                hnet_child_optim.zero_grad()

                if i % 100 == 99:
                    acc = (y_hat.argmax(dim=-1) == y.argmax(dim=-1)).float().mean() * 100.
                    print_metrics(
                        datasets_for_eval, hnet_root, hnet_child, solver_root, solver_child,
                        prefix=f"[{p_i + 1}:{phase} | {d_i}/{len(data_handlers)} | {epoch + 1}/{config['epochs']} | {i + 1}]",
                        skip_phases=[],
                        wandb_run=wandb_run, additional_metrics={
                            "loss_class": loss_class.item(),
                            "acc_class": acc,
                            "loss_solver_params_reg": loss_solver_params_reg.item(),
                            "loss_reg": loss_reg.item(),
                        }
                    )
                    print("---")
                    log_step += 1

### Manual analysis

In [ ]:
m_y = mnist.output_to_torch_tensor(m_y, config["device"], mode="train")

In [ ]:
y_hat, params_solver = infer(m_X, "hnet->hnet->solver", hnet_root_cond_id=2, hnet_child_cond_id=0, hnet_root=hnet_root, hnet_child=hnet_child, solver_root=solver_root, solver_child=solver_child)

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
print_metrics(
    {"MNIST": (0, 2, 0, mnist)},
    hnet_root, hnet_child, solver_root, solver_child,
    prefix=f"[{phase} | {epoch}/{config['epochs']} | {i + 1}]\nM: {m_loss_reg:.3f} F: {f_loss_reg:.3f}",
    skip_phases=["hnet->hnet->solver"] if p_i == 0 and phase == "hnet->solver" else [],
)

In [ ]:
p_solver = hnet_root(cond_id=0)
y_h = solver_root.forward(test_in[:32], weights=correct_param_shapes(solver_root, p_solver))

In [ ]:
# test_in = mnist.input_to_torch_tensor( \
#     mnist.get_test_inputs(), config["device"], mode='inference')
# test_out = mnist.input_to_torch_tensor( \
#     mnist.get_test_outputs(), config["device"], mode='inference')
# test_lbls = test_out.max(dim=1)[1]

s, e = 6000,6500
y_hat, params_solver = infer(test_in[s:e], "hnet->solver", hnet_root_cond_id=0, hnet_child_cond_id=None, hnet_root=hnet_root, hnet_child=hnet_child, solver_root=solver_root, solver_child=solver_child)
print("MNIST test accuracy:", (y_hat.max(dim=1)[1] == test_out[s:e].max(dim=1)[1]).float().mean())

# logits = solver_root(test_in, weights=correct_param_shapes(solver_root, params_solver))
# pred_lbls = logits.max(dim=1)[1]

# torch.sum(test_lbls == pred_lbls) / test_lbls.numel() * 100.

- 98 mnist, 89 fmnist (hypernet -> target net)
- 98 mnist, 88 fmnist (hypernet -> hypernet -> target net)